In [1]:
import pandas as pd
import random
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, average_precision_score)
random.seed(42)

In [2]:
df = pd.read_csv('/Volumes/Seagate External HD 1/Independent Study/NYTRestaurantReviews/data/classificationdf_final.csv')
df=df.iloc[:,2:]
df.head()

/var/folders/06/rlq1jslj0qsdfph0bh6cp1mc0000gn/T/ipykernel_48340/2490038141.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Volumes/Seagate External HD 1/Independent Study/NYTRestaurantReviews/data/classificationdf_final.csv')


,Restaurant_name,zipcode,Cuisine_Category,Wealthy_Zip,Median Income,Population,English,Spanish,Indo-European,Asian-Pacific,...,shape_leng,shape_area,opening_hours_x,distance_to_park_x,distance_to_subway_x,distance_to_bus_stop_x,opening_hours_y,distance_to_park_y,distance_to_subway_y,distance_to_bus_stop_y
0,'WICHCRAFT,10016,Other,1,145864.0,54369.0,36846.0,5240.0,4180.0,1060.0,...,360037.634094,6.366460e+08,Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00...,682.447681,373.335318,333.792633,Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00...,682.447681,373.335318,333.792633
1,12 CHAIRS,10012,Other,0,126890.0,21216.0,15813.0,1222.0,855.0,367.0,...,360037.634094,6.366460e+08,NaN,544.500976,541.564337,373.767984,NaN,544.500976,541.564337,373.767984
2,15 EAST,10003,Asian,1,152863.0,53877.0,40948.0,3278.0,2489.0,789.0,...,360037.634094,6.366460e+08,Mo-Fr 11:30-22:30; Sa 14:00-22:30; Su 13:00-22:00,369.193375,388.426032,113.713747,Mo-Fr 11:30-22:30; Sa 14:00-22:30; Su 13:00-22:00,369.193375,388.426032,113.713747
3,188 BAKERY CUCHIFRITOS,10468,Other,0,43985.0,81397.0,15410.0,53330.0,25403.0,27927.0,...,463180.578868,1.187175e+09,NaN,112.880520,36.685305,485.761214,NaN,112.880520,36.685305,485.761214
4,2ND CITY,10014,Other,1,147267.0,29461.0,24073.0,1876.0,1699.0,177.0,...,360037.634094,6.366460e+08,NaN,286.769836,336.354459,572.074009,NaN,286.769836,336.354459,572.074009


In [3]:
columns_to_keep = ['Restaurant_name',
 'Cuisine_Category',
 'Wealthy_Zip',
 'Median Income',
 'Population',
 'English',
 'Spanish',
 'Indo-European',
 'Asian-Pacific',
 'Other Languages',
 'Total 25+',
 'Bachelor',
 'Master',
 'Professional',
 'College Graduates (%)',
 'Most Common Language (Other than English)',
 'Reviewed by NYT',
 'boro_name',
 'distance_to_park_x',
 'distance_to_subway_x',
 'distance_to_bus_stop_x',
]

columns_to_rename = {'opening_hours_x':'opening_hours',
                     'distance_to_park_x':'distance_to_park',
                     'distance_to_subway_x':'distance_to_subway',
                     'distance_to_bus_stop_x':'distance_to_bus_stop'}


df2 = df[columns_to_keep]
df3 = df2.rename(columns = columns_to_rename)
df3.set_index('Restaurant_name', inplace = True)
print(df3.shape)
df3.head()

(19242, 20)


,Cuisine_Category,Wealthy_Zip,Median Income,Population,English,Spanish,Indo-European,Asian-Pacific,Other Languages,Total 25+,Bachelor,Master,Professional,College Graduates (%),Most Common Language (Other than English),Reviewed by NYT,boro_name,distance_to_park,distance_to_subway,distance_to_bus_stop
Restaurant_name,,,,,,,,,,,,,,,,,,,,
'WICHCRAFT,Other,1,145864.0,54369.0,36846.0,5240.0,4180.0,1060.0,871.0,43500.0,18102.0,10877.0,4803.0,0.776598,Spanish,1,Manhattan,682.447681,373.335318,333.792633
12 CHAIRS,Other,0,126890.0,21216.0,15813.0,1222.0,855.0,367.0,450.0,16809.0,8292.0,3089.0,1217.0,0.749479,Spanish,1,Manhattan,544.500976,541.564337,373.767984
15 EAST,Asian,1,152863.0,53877.0,40948.0,3278.0,2489.0,789.0,1757.0,37093.0,16265.0,7846.0,3915.0,0.755560,Spanish,1,Manhattan,369.193375,388.426032,113.713747
188 BAKERY CUCHIFRITOS,Other,0,43985.0,81397.0,15410.0,53330.0,25403.0,27927.0,773.0,52353.0,5028.0,2119.0,587.0,0.147728,Spanish,1,Bronx,112.880520,36.685305,485.761214
2ND CITY,Other,1,147267.0,29461.0,24073.0,1876.0,1699.0,177.0,726.0,25527.0,12251.0,6490.0,1836.0,0.806088,Spanish,1,Manhattan,286.769836,336.354459,572.074009


In [4]:
categorical_df = pd.get_dummies(df3.select_dtypes(include = 'object'))
categorical_df.head()

,Cuisine_Category_African,Cuisine_Category_American,Cuisine_Category_Asian,Cuisine_Category_European,Cuisine_Category_Latin American,Cuisine_Category_Middle Eastern,Cuisine_Category_Other,Most Common Language (Other than English)_Other Languages,Most Common Language (Other than English)_Spanish,boro_name_Bronx,boro_name_Brooklyn,boro_name_Manhattan,boro_name_Queens,boro_name_Staten Island
Restaurant_name,,,,,,,,,,,,,,
'WICHCRAFT,0,0,0,0,0,0,1,0,1,0,0,1,0,0
12 CHAIRS,0,0,0,0,0,0,1,0,1,0,0,1,0,0
15 EAST,0,0,1,0,0,0,0,0,1,0,0,1,0,0
188 BAKERY CUCHIFRITOS,0,0,0,0,0,0,1,0,1,1,0,0,0,0
2ND CITY,0,0,0,0,0,0,1,0,1,0,0,1,0,0


In [5]:
numerical_df = df3.select_dtypes(include='float64').fillna(999999)
numerical_df.head()

,Median Income,Population,English,Spanish,Indo-European,Asian-Pacific,Other Languages,Total 25+,Bachelor,Master,Professional,College Graduates (%),distance_to_park,distance_to_subway,distance_to_bus_stop
Restaurant_name,,,,,,,,,,,,,,,
'WICHCRAFT,145864.0,54369.0,36846.0,5240.0,4180.0,1060.0,871.0,43500.0,18102.0,10877.0,4803.0,0.776598,682.447681,373.335318,333.792633
12 CHAIRS,126890.0,21216.0,15813.0,1222.0,855.0,367.0,450.0,16809.0,8292.0,3089.0,1217.0,0.749479,544.500976,541.564337,373.767984
15 EAST,152863.0,53877.0,40948.0,3278.0,2489.0,789.0,1757.0,37093.0,16265.0,7846.0,3915.0,0.755560,369.193375,388.426032,113.713747
188 BAKERY CUCHIFRITOS,43985.0,81397.0,15410.0,53330.0,25403.0,27927.0,773.0,52353.0,5028.0,2119.0,587.0,0.147728,112.880520,36.685305,485.761214
2ND CITY,147267.0,29461.0,24073.0,1876.0,1699.0,177.0,726.0,25527.0,12251.0,6490.0,1836.0,0.806088,286.769836,336.354459,572.074009


In [6]:
int_df = df3.select_dtypes(include='int64').drop(columns = ['Reviewed by NYT'])
int_df

,Wealthy_Zip
Restaurant_name,
'WICHCRAFT,1
12 CHAIRS,0
15 EAST,1
188 BAKERY CUCHIFRITOS,0
2ND CITY,1
...,...
MISS KOREA,0
UPWELL,0
CHARLEY'S,0


In [7]:
Target = df3['Reviewed by NYT']

# Model Evaluation

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, average_precision_score)

# ---------------------------------
# 1. Create X and Y variables
# ---------------------------------
# X is built by concatenating int_df, categorical_df, and numerical_df.
X = pd.concat([int_df, categorical_df, numerical_df], axis=1)
y = Target  # Target variable

# ---------------------------------
# 2. Create Train/Test/Validation Splits
# ---------------------------------
# First, split into train (70%) and temp (30%) with stratification.
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

# Then split the temporary set equally into test (15%) and validation (15%).
X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

# ---------------------------------
# 3. Preprocessing: Scaling Numerical Variables
# ---------------------------------
# Assume numerical_df contains your float variables.
numerical_cols = numerical_df.columns.tolist()

# StandardScaler will be applied only to numerical columns.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)
    ],
    remainder='passthrough'
)

# ---------------------------------
# 4. Define Models and Build Pipelines
# ---------------------------------
# Define the classifiers.
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'SVM': SVC(probability=True, random_state=42)
}

# For models that require scaling (e.g., linear models) we include the preprocessor.
# We'll apply StandardScaler only for Logistic Regression and SVM.
scaling_models = ['Logistic Regression', 'SVM']

pipelines = {}
for name, clf in models.items():
    if name in scaling_models:
        # Pipeline with StandardScaler applied to numerical columns.
        pipelines[name] = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', clf)
        ])
    else:
        # For tree-based models, we skip scaling.
        pipelines[name] = Pipeline(steps=[
            ('classifier', clf)
        ])

# ---------------------------------
# 5. Evaluation Function
# ---------------------------------
def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    # Use predict_proba if available.
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X)[:, 1]
    else:
        y_proba = None

    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred, zero_division=0)
    rec = recall_score(y, y_pred, zero_division=0)
    f1 = f1_score(y, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y, y_proba) if y_proba is not None else np.nan
    pr_auc = average_precision_score(y, y_proba) if y_proba is not None else np.nan

    return {
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

# ---------------------------------
# 6. Train Each Model and Evaluate on Each Split
# ---------------------------------
results = {}

for name, pipeline in pipelines.items():
    # Train the model.
    pipeline.fit(X_train, y_train)
    
    # Evaluate on train, test, and validation sets.
    results[name] = {
        'Train': evaluate_model(pipeline, X_train, y_train),
        'Test': evaluate_model(pipeline, X_test, y_test),
        'Validation': evaluate_model(pipeline, X_val, y_val)
    }

# Convert the results dictionary into a single DataFrame.
rows = []
for model_name, splits in results.items():
    row = {'Model': model_name}
    for split_name, metrics in splits.items():
        for metric_name, value in metrics.items():
            row[f'{split_name}_{metric_name}'] = value
    rows.append(row)

metrics_df = pd.DataFrame(rows)

# Display the performance metrics grid.
metrics_df

,Model,Train_Accuracy,Train_Precision,Train_Recall,Train_F1 Score,Train_ROC-AUC,Train_PR-AUC,Test_Accuracy,Test_Precision,Test_Recall,Test_F1 Score,Test_ROC-AUC,Test_PR-AUC,Validation_Accuracy,Validation_Precision,Validation_Recall,Validation_F1 Score,Validation_ROC-AUC,Validation_PR-AUC
0,Logistic Regression,0.962284,1.0,0.415420,0.586992,0.806437,0.550784,0.961885,1.000000,0.408602,0.580153,0.817812,0.543121,0.967094,1.000000,0.491979,0.659498,0.832917,0.621071
1,Decision Tree,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.940748,0.539267,0.553763,0.546419,0.760585,0.327386,0.937305,0.513514,0.609626,0.557457,0.784813,0.338337
2,Random Forest,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.962231,0.866667,0.489247,0.625430,0.882175,0.659723,0.966747,0.876033,0.566845,0.688312,0.888263,0.707275
3,XGBoost,0.987230,1.0,0.802071,0.890166,0.999552,0.994634,0.963964,0.927083,0.478495,0.631206,0.921338,0.684084,0.970558,0.918033,0.598930,0.724919,0.905575,0.724962
4,SVM,0.962135,1.0,0.413119,0.584691,0.731784,0.503111,0.961885,1.000000,0.408602,0.580153,0.721384,0.480655,0.966747,1.000000,0.486631,0.654676,0.784957,0.584479


In [13]:
# Combine X and y for each split
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)
validation = pd.concat([X_val, y_val], axis=1)

# Define the directory path where you want to save the files
save_path = '/Volumes/Seagate External HD 1/Independent Study/NYTRestaurantReviews/data/modeling'

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save each dataset to a CSV file without the index
train.to_csv(os.path.join(save_path, 'train.csv'), index=True)
test.to_csv(os.path.join(save_path, 'test.csv'), index=True)
validation.to_csv(os.path.join(save_path, 'validation.csv'), index=True)